In [12]:
using HopTB, Test
include("zoo.jl")

let
    lat = [(√3) / 2 (√3) / 2 0; -1 / 2 1 / 2 0; 0 0 1]
    BN = TBModel(2, lat)
    sethopping!(BN, [0, 0, 0], 1, 1, -1 / 2)
    sethopping!(BN, [0, 0, 0], 2, 2, 1 / 2)
    sethopping!(BN, [0, 0, 0], 1, 2, -1)
    sethopping!(BN, [-1, 0, 0], 1, 2, -1)
    sethopping!(BN, [0, -1, 0], 1, 2, -1)
    setposition!(BN, [0, 0, 0], 1, 1, 1, (BN.lat * [1 / 3, 1 / 3, 0])[1])
    setposition!(BN, [0, 0, 0], 1, 1, 2, (BN.lat * [1 / 3, 1 / 3, 0])[2])
    setposition!(BN, [0, 0, 0], 1, 1, 3, (BN.lat * [1 / 3, 1 / 3, 0])[3])
    setposition!(BN, [0, 0, 0], 2, 2, 1, (BN.lat * [2 / 3, 2 / 3, 0])[1])
    setposition!(BN, [0, 0, 0], 2, 2, 2, (BN.lat * [2 / 3, 2 / 3, 0])[2])
    setposition!(BN, [0, 0, 0], 2, 2, 3, (BN.lat * [2 / 3, 2 / 3, 0])[3])

    @test HopTB.Hall._getahc(BN, 1, 2, [1 / 3 2 / 3 0; 1 / 3 2 / 3 0]')[1] ≈ -3.0 atol = 1.0e-7
    @test HopTB.Hall.getahc(BN, 1, 2, [4, 4, 1])[1] ≈ 0.0 atol = 1.0e-7
    @test HopTB.Hall.collect_berry_curvature(BN, 1, 2, [1 / 3 2 / 3 0; 1 / 3 2 / 3 0]') ≈ [-1.5 -1.5; 1.5 1.5] atol = 1.0e-7
end

let
    tm = get_Kane_Mele()
    @test HopTB.Hall.getshc(tm, 1, 2, 3, [10, 10, 1])[1, 1] ≈ -393.2616 atol=1.0e-3
end

@register "Fe.openmx" tm->begin
    ntm = HopTB.change_energy_reference(tm, -5.441778786050317507)
    σs = HopTB.Hall.getahc(ntm, 1, 2, [10, 10, 10])
    @test σs[1, 1] ≈ -794.815 atol=1.0e-3
end


Precompiling HopTB
        Info Given HopTB was explicitly requested, output will be shown live 
ERROR: LoadError: ParseError:
# Error @ ]8;;file://C:/Users/woxin/Documents/GitHub/HopTB.jl/src/hall.jl#300:2\C:\Users\woxin\Documents\GitHub\HopTB.jl\src\hall.jl:300:2]8;;\
end

#└ ── Expected `end`
Stacktrace:
 [1] top-level scope
   @ C:\Users\woxin\Documents\GitHub\HopTB.jl\src\hall.jl:300
 [2] include(mod::Module, _path::String)
   @ Base .\Base.jl:495
 [3] include(x::String)
   @ HopTB C:\Users\woxin\Documents\GitHub\HopTB.jl\src\HopTB.jl:1
 [4] top-level scope
   @ C:\Users\woxin\Documents\GitHub\HopTB.jl\src\HopTB.jl:22
 [5] include
   @ .\Base.jl:495 [inlined]
 [6] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt128}}, source::Nothing)
   @ Base .\loading.jl:2222
 [7] top-level scope
   @ stdin:3
in expression starting at C:\Users\woxin\Docu

LoadError: The following 1 direct dependency failed to precompile:

HopTB [3804c90a-d6ae-434d-aed1-763ec3185202]

Failed to precompile HopTB [3804c90a-d6ae-434d-aed1-763ec3185202] to "C:\\Users\\woxin\\.julia\\compiled\\v1.10\\HopTB\\jl_7961.tmp".
[91m[1mERROR: [22m[39mLoadError: ParseError:
[90m# Error @ [0;0m]8;;file://C:/Users/woxin/Documents/GitHub/HopTB.jl/src/hall.jl#300:2\[90mC:\Users\woxin\Documents\GitHub\HopTB.jl\src\hall.jl:300:2[0;0m]8;;\
end

[48;2;120;70;70m[0;0m[90m#└ ── [0;0m[91mExpected `end`[0;0m
Stacktrace:
 [1] top-level scope
[90m   @[39m [90mC:\Users\woxin\Documents\GitHub\HopTB.jl\src\[39m[90m[4mhall.jl:300[24m[39m
 [2] [0m[1minclude[22m[0m[1m([22m[90mmod[39m::[0mModule, [90m_path[39m::[0mString[0m[1m)[22m
[90m   @[39m [90mBase[39m [90m.\[39m[90m[4mBase.jl:495[24m[39m
 [3] [0m[1minclude[22m[0m[1m([22m[90mx[39m::[0mString[0m[1m)[22m
[90m   @[39m [35mHopTB[39m [90mC:\Users\woxin\Documents\GitHub\Ho